# A proxy to assess scaling
To assess scaling, we consider the injection protocol for Grimsel Test Site, and the corresponding pressure values at the injection point. That is, for a given injection rate, we should expect that our model is able to reproduce a pressure (in the injection cell) that is similar to the pressure reported in the publications.

### Code setup
We are working on a custom changelist that sets the time step to `40 minutes`, with duration `[0, 40 minutes]`. The injection rate is set to a constant amount of `20 l/min`.

### Parameter setup
We include the shearzone `S1_2` and conduct injection to the borehole `INJ1`.

We turn off all gravity effects. This means that:
* No source terms (scalar and mechanics)
* Scalar dirichlet set to 0
* Mechanics neumann set to 0 except at 3 points on bottom of domain



### Expected results
From the literature (Doetsch et al 2019), for injection rates of `[10, 25] l/min`, we expect a pressure response to be in the range `[5, 9] MPa`.

In [1]:
import sys
sys.path.extend([
    '/home/haakon/mastersproject', 
    '/home/haakon/mastersproject/src/mastersproject'
])

import GTS as gts
from GTS.isc_modelling.setup import run_abstract_model
import porepy as pp
import numpy as np

/home/haakon/porepy/src/porepy/viz/exporter.py:928: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))


In [2]:
# Prepare parameterse for run_abstract_model
model = gts.ContactMechanicsBiotISC
run_model_method = pp.run_time_dependent_model

params = {
    "length_scale": .1,
    "scalar_scale": pp.GIGA,
    "shearzone_names": ["S1_2"]
}

In [3]:
# Run the model
setup = run_abstract_model(
    model=model,
    run_model_method=run_model_method,
    params=params,
)

INFO:GTS.isc_modelling.setup:Preparing setup for simulation on 2020-03-31T14:27:21+02:00
INFO:GTS.isc_modelling.setup:Simulation parameters:
 {'bounding_box': {'xmax': 80,
                  'xmin': -20,
                  'ymax': 150,
                  'ymin': 50,
                  'zmax': 75,
                  'zmin': -25},
 'folder_name': PosixPath('/home/haakon/mastersproject/src/mastersproject/GTS/isc_modelling/results/200331/default/default_1'),
 'length_scale': 0.1,
 'mesh_args': {'mesh_size_bound': 60,
               'mesh_size_frac': 10,
               'mesh_size_min': 1.0},
 'scalar_scale': 1000000000.0,
 'shearzone_names': ['S1_2'],
 'solver': 'direct',
 'source_scalar_borehole_shearzone': {'borehole': 'INJ1', 'shearzone': 'S1_2'},
 'stress': array([[-11393046.22034133,    597621.26708787,   1842573.57669909],
       [   597621.26708787,  -8873267.78285116,   -586150.35104884],
       [  1842573.57669909,   -586150.35104884, -10733685.99680752]])}
INFO:GTS.isc_modelling.contac

INFO:GTS.isc_modelling.mechanics:Finished 'create_grid' in 1.3370 secs
INFO:GTS.isc_modelling.mechanics:Finished 'create_grid' in 1.3370 secs
INFO:GTS.isc_modelling.mechanics:'create_grid' returned None
INFO:GTS.isc_modelling.mechanics:'create_grid' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Tagging grid of name: S1_2, and dimension 2
INFO:GTS.isc_modelling.contact_mechanics_biot:Tagging grid of name: S1_2, and dimension 2
INFO:GTS.isc_modelling.contact_mechanics_biot:Setting non-zero source value for pressure
INFO:GTS.isc_modelling.contact_mechanics_biot:Setting non-zero source value for pressure
INFO:GTS.isc_modelling.contact_mechanics_biot:Closest cell found has distance: 20.557855
INFO:GTS.isc_modelling.contact_mechanics_biot:Closest cell found has distance: 20.557855
INFO:porepy.models.contact_mechanics_biot_model:Discretize
INFO:porepy.models.contact_mechanics_biot_model:Discretize
INFO:porepy.numerics.fv.mpsa:Split MPSA discretization into 1 parts
INFO:porepy.nu

INFO:GTS.isc_modelling.contact_mechanics_biot:diff iter u = 5.762930e-06
INFO:GTS.isc_modelling.contact_mechanics_biot:diff iter u = 5.762930e-06
INFO:GTS.isc_modelling.contact_mechanics_biot:diff init u = 1.702807e-01
INFO:GTS.isc_modelling.contact_mechanics_biot:diff init u = 1.702807e-01
INFO:GTS.isc_modelling.contact_mechanics_biot:diff iter contact = 1.113396e+17
INFO:GTS.isc_modelling.contact_mechanics_biot:diff iter contact = 1.113396e+17
INFO:GTS.isc_modelling.contact_mechanics_biot:diff init contact = 4.239128e+15
INFO:GTS.isc_modelling.contact_mechanics_biot:diff init contact = 4.239128e+15
INFO:GTS.isc_modelling.contact_mechanics_biot:diff iter scalar = 4.758050e+12
INFO:GTS.isc_modelling.contact_mechanics_biot:diff iter scalar = 4.758050e+12
INFO:GTS.isc_modelling.contact_mechanics_biot:diff init scalar = 1.965433e+16
INFO:GTS.isc_modelling.contact_mechanics_biot:diff init scalar = 1.965433e+16
INFO:GTS.isc_modelling.contact_mechanics_biot:Error in contact force is 2.626474

In [4]:
f = setup.gb.grids_of_dimension(2)[0]
df = setup.gb.node_props(f)

g = setup.gb.grids_of_dimension(3)[0]
d = setup.gb.node_props(g)

In [5]:
state=df['state']
well = state['well']
loc = np.where(well)[0]
state['p_exp'][loc] / pp.MEGA

array([98.48889702])

In [32]:
state=df['state']
well = state['well']
loc = np.where(well)[0]
state['p_exp'][loc] / pp.MEGA

array([106.47070393])

In [36]:
state=df['state']
well = state['well']
loc = np.where(well)[0]
state['p_exp'][loc] / pp.MEGA

array([98.48821998])

In [37]:
setup.gb

Grid bucket containing 2 grids:
1 grids of dimension 3
1 grids of dimension 2

In [44]:
p_exp2 = state['p_exp'] / pp.MEGA
np.sort(p_exp2)

array([ 4.40767671,  8.58911425,  8.81202665,  9.33535998,  9.59605464,
       10.29953118, 10.32956062, 10.38993816, 10.54576518, 10.5848433 ,
       10.64216312, 10.68274871, 10.71203841, 10.74731369, 10.75837069,
       10.80083865, 10.80693201, 10.80974007, 10.86908971, 10.89234737,
       10.94464609, 10.9707235 , 10.97463278, 11.00130669, 11.00272508,
       11.05820776, 11.06385637, 11.06570477, 11.08943185, 11.09274676,
       11.15605645, 11.17980071, 11.18157882, 11.19180413, 11.19406144,
       11.19910787, 11.19938549, 11.20340204, 11.20909583, 11.27627384,
       11.32844804, 11.33204887, 12.43846052, 98.48821998])

In [47]:
# 3d Grid
state3 = d['state']
p_exp3 = state3['p_exp'] / pp.MEGA
np.where(p_exp3>30)[0]

array([11, 37])

In [49]:
g.cell_centers[:, np.where(p_exp3>30)[0]] * .1

array([[ 44.32205995,  39.88655706],
       [105.74178145, 111.58983313],
       [ 12.2387516 ,  10.64923882]])

In [55]:
well = f.cell_centers[:, loc]
well

array([[ 403.06083064],
       [1075.03563715],
       [ 126.75489484]])

In [56]:
ids, dsts = g.closest_cell(well, return_distance=True)
(ids, dsts)

(array([11]), array([44.07117768]))

In [ ]:
np.norm()